# Datathink 2023
Roma 2023. Biblioteca Hertziana + DSV@UZH + Max Planck Society
# Sensing data fusion
This notebook illustrates how to combine and aggregates different types of data collected from different sensors:
- GPS tracks from location devices (i.e. smartphones, smartwatches, activity trackers, etc.)
- Gas / air quality data from the CoCi's CoSense unit by COSS@ETHZ
- GQ multimeter for electric+electromagnetic+radiofrequency

In [99]:
import pandas as pd
import os
pd.set_option('display.max_rows', 200)

# Importing GPS tracks

## method 2 with gpxo

In [100]:
import gpxo
paths = []
folder = os.getcwd()
for file in sorted(os.listdir(folder)):
    if file.endswith(".gpx"):
        paths.append(os.path.join(folder, file))
print(sorted(paths))

df_gps = pd.DataFrame()
for gps_tracks in paths:
    gps_data = gpxo.Track(gps_tracks)
    df_gps_temp = gps_data.data
    df_gps = pd.concat([df_gps, df_gps_temp])
df_gps

['/mnt/c/Users/jaargota/Documents/202302-Spring 2023/20230227-Datathink_Rome/test_zh_20230225/activity_10574965994_02.gpx', '/mnt/c/Users/jaargota/Documents/202302-Spring 2023/20230227-Datathink_Rome/test_zh_20230225/activity_10582299864_01.gpx']


,latitude (°),longitude (°),distance (km),compass (°),duration (s),velocity (km/h),elevation (m)
time,,,,,,,
2023-02-25 15:44:59,47.369200,8.541591,0.000000,180.000000,0.0,0.097527,413.000000
2023-02-25 15:45:10,47.369197,8.541591,0.000298,172.153726,11.0,0.097233,413.000000
2023-02-25 16:08:06,47.368997,8.541674,0.023371,145.267669,1387.0,8.158359,412.600006
2023-02-25 16:08:10,47.368949,8.541771,0.032462,62.068828,1391.0,7.618980,411.399994
2023-02-25 16:08:16,47.369051,8.541766,0.043753,4.242156,1397.0,5.732143,411.000000
...,...,...,...,...,...,...,...
2023-02-25 15:43:23,47.369171,8.541697,3.826645,195.568910,2472.0,4.134000,411.600006
2023-02-25 15:43:29,47.369119,8.541695,3.832425,249.486886,2478.0,2.945700,411.799988
2023-02-25 15:43:35,47.369146,8.541659,3.836464,313.906654,2484.0,2.072700,412.200012


In [101]:
df_gps.index

DatetimeIndex(['2023-02-25 15:44:59', '2023-02-25 15:45:10',
               '2023-02-25 16:08:06', '2023-02-25 16:08:10',
               '2023-02-25 16:08:16', '2023-02-25 16:08:23',
               '2023-02-25 16:08:29', '2023-02-25 16:08:35',
               '2023-02-25 16:08:38', '2023-02-25 16:08:40',
               ...
               '2023-02-25 15:43:05', '2023-02-25 15:43:08',
               '2023-02-25 15:43:10', '2023-02-25 15:43:13',
               '2023-02-25 15:43:15', '2023-02-25 15:43:23',
               '2023-02-25 15:43:29', '2023-02-25 15:43:35',
               '2023-02-25 15:43:41', '2023-02-25 15:43:47'],
              dtype='datetime64[ns]', name='time', length=1397, freq=None)

In [102]:
# read Datetime Index in GMT, convert to CET and remove timezone while preserving local time
df_gps["time_CET"] = df_gps.index.tz_localize("GMT").tz_convert('CET').tz_localize(None)
df_gps= df_gps.set_index("time_CET")
df_gps = df_gps.sort_index(ascending=True)
df_gps

,latitude (°),longitude (°),distance (km),compass (°),duration (s),velocity (km/h),elevation (m)
time_CET,,,,,,,
2023-02-25 16:02:11,47.400237,8.543939,0.000000,304.133128,0.0,3.340350,473.000000
2023-02-25 16:02:19,47.400274,8.543857,0.007423,269.837538,8.0,4.244656,473.000000
2023-02-25 16:02:28,47.400207,8.543714,0.020578,234.862841,17.0,5.445600,473.399994
2023-02-25 16:02:30,47.400191,8.543681,0.023626,232.005455,19.0,4.886400,473.399994
2023-02-25 16:02:31,47.400184,8.543668,0.024900,232.959085,20.0,4.665200,473.399994
...,...,...,...,...,...,...,...
2023-02-25 18:42:00,47.400325,8.543864,5.638714,43.688339,7021.0,4.210070,472.799988
2023-02-25 18:42:07,47.400373,8.543921,5.645556,17.474280,7028.0,3.205052,472.600006
2023-02-25 18:42:11,47.400403,8.543918,5.648918,24.963792,7032.0,2.221800,472.600006


In [103]:
gps_data.map(embed=True)

/home/jaargota/.conda/envs/py310/lib/python3.10/site-packages/IPython/core/display.py:419: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## resampling to seconds and interpolating

In [91]:
df_gps = df_gps.resample("s").interpolate("linear")
df_gps

,latitude (°),longitude (°),distance (km),compass (°),duration (s),velocity (km/h),elevation (m)
time_CET,,,,,,,
2023-02-25 16:02:11,47.400237,8.543939,0.000000,304.133128,0.0,3.340350,473.000000
2023-02-25 16:02:12,47.400241,8.543929,0.000928,299.846179,1.0,3.453388,473.000000
2023-02-25 16:02:13,47.400246,8.543919,0.001856,295.559231,2.0,3.566426,473.000000
2023-02-25 16:02:14,47.400251,8.543908,0.002784,291.272282,3.0,3.679465,473.000000
2023-02-25 16:02:15,47.400255,8.543898,0.003711,286.985333,4.0,3.792503,473.000000
...,...,...,...,...,...,...,...
2023-02-25 18:42:15,47.400408,8.543934,5.650281,83.682297,7036.0,1.226700,472.600006
2023-02-25 18:42:16,47.400408,8.543936,5.650457,87.425441,7037.0,1.078425,472.600006
2023-02-25 18:42:17,47.400407,8.543938,5.650633,91.168585,7038.0,0.930150,472.600006


# Getting gas sensor data

In [92]:
time_correction_factor_gas = pd.Timedelta(hours=0, minutes=0, seconds=0)

df_gas = pd.read_csv("data_raspberry8.csv", sep=",", index_col="Date/Time")
df_gas["time"] = pd.to_datetime(df_gas.index) - time_correction_factor_gas
df_gas = df_gas.set_index("time")
df_gas

,Temperature (C),Humidity (%),PM1 (ug/m3),PM2.5 (ug/m3),PM10 (ug/m3)
time,,,,,
2023-02-23 17:26:09,22.136909,37.086529,5.40,13.95,21.96
2023-02-23 17:28:25,23.042645,36.138682,2.95,5.08,6.99
2023-02-23 17:29:19,23.277199,35.946182,4.77,13.23,21.18
2023-02-23 17:30:22,23.881952,35.713894,2.59,2.73,2.73
2023-02-23 17:31:19,23.525144,35.504396,6.00,14.37,22.16
...,...,...,...,...,...
2023-02-25 22:40:54,16.405332,63.338851,4.04,8.67,12.94
2023-02-25 22:41:20,16.834131,61.948389,3.83,10.71,17.19
2023-02-25 22:42:19,17.732733,61.533585,1.95,2.06,2.06


## grouping per time (index) for avoiding errors due to duplicated indexes

In [93]:
df_gas = df_gas.groupby(by="time", as_index=True).agg("mean")

## resampling to seconds and interpolating

In [94]:
df_gas = df_gas.resample("s").interpolate("linear")
df_gas

,Temperature (C),Humidity (%),PM1 (ug/m3),PM2.5 (ug/m3),PM10 (ug/m3)
time,,,,,
2023-02-23 17:26:09,22.136909,37.086529,5.400000,13.950000,21.960000
2023-02-23 17:26:10,22.143569,37.079559,5.381985,13.884779,21.849926
2023-02-23 17:26:11,22.150229,37.072590,5.363971,13.819559,21.739853
2023-02-23 17:26:12,22.156889,37.065620,5.345956,13.754338,21.629779
2023-02-23 17:26:13,22.163549,37.058651,5.327941,13.689118,21.519706
...,...,...,...,...,...
2023-02-25 22:44:21,19.442177,56.297954,1.481212,4.743030,7.842727
2023-02-25 22:44:22,19.452269,56.273171,1.455909,4.672273,7.729545
2023-02-25 22:44:23,19.462360,56.248387,1.430606,4.601515,7.616364


# Getting GQ data

In [95]:
# in test file emfhistory_02242023_145731.csv last timestamp is 2023/02/24 18:57:09, but file was writen at 15:06:39. Shift: 3:50:30
time_correction_factor_GQ = pd.Timedelta(hours=3, minutes=50, seconds=30) 

df_GQ = pd.read_csv("emfhistory_02252023_225547.csv", sep=",", header=[2])
names_cols_GQ = {"mG":"EMF(mG)", 
                 "V/m": "EF(V/m)", 
                 "mW/m2": "RF Power Density(mW/m2)", 
                 "mW/cm2": "RF Power Density(mW/cm2)", 
                 "pW/cm2": "RF Power Density(pW/cm2)"}
df_GQ.rename(columns=names_cols_GQ)

df_GQ["time"] = pd.to_datetime(df_GQ["Date and Time"])# - time_correction_factor_GQ
df_GQ = df_GQ.set_index("time")
df_GQ

,Date and Time,mG,V/m,mW/m2,mW/cm2,pW/cm2,Possible Source
time,,,,,,,
2021-09-09 11:37:54,2021/09/09 11:37:54,1.4,31.5,2.291,0.000,229100.438,Mixed
2021-09-09 11:37:55,2021/09/09 11:37:55,1.4,31.5,2.291,0.000,229100.438,Mixed
2021-09-09 11:37:56,2021/09/09 11:37:56,1.4,1.4,1.656,0.000,165598.000,Mixed
2021-09-09 11:37:57,2021/09/09 11:37:57,1.4,1.4,1.656,0.000,165598.000,WiFi/Phone
2021-09-09 11:37:58,2021/09/09 11:37:58,1.4,1.2,1.116,0.000,111585.555,WiFi/Phone
...,...,...,...,...,...,...,...
2023-02-25 23:10:05,2023/02/25 23:10:05,0.5,83.1,7.872,0.001,787163.438,Mixed
2023-02-25 23:10:06,2023/02/25 23:10:06,0.5,84.1,7.516,0.001,751589.000,Mixed
2023-02-25 23:10:07,2023/02/25 23:10:07,0.5,80.7,6.644,0.001,664381.250,Mixed


# merging

In [96]:
df_merge = df_gps.merge(df_gas, how="left", left_index=True, right_index=True).merge(df_GQ, left_index=True, right_index=True)

In [97]:
df_merge.head(10)

,latitude (°),longitude (°),distance (km),compass (°),duration (s),velocity (km/h),elevation (m),Temperature (C),Humidity (%),PM1 (ug/m3),PM2.5 (ug/m3),PM10 (ug/m3),Date and Time,mG,V/m,mW/m2,mW/cm2,pW/cm2,Possible Source
2023-02-25 16:02:11,47.400237,8.543939,0.000000,304.133128,0.0,3.340350,473.000000,21.451999,30.158297,3.562000,9.880667,15.826000,2023/02/25 16:02:11,1.2,0.8,0.092,0.0,9227.629,Mixed
2023-02-25 16:02:12,47.400241,8.543929,0.000928,299.846179,1.0,3.453388,473.000000,21.421433,30.180145,3.605500,10.044333,16.104000,2023/02/25 16:02:12,1.4,0.4,0.146,0.0,14580.254,Mixed
2023-02-25 16:02:13,47.400246,8.543919,0.001856,295.559231,2.0,3.566426,473.000000,21.390866,30.201992,3.649000,10.208000,16.382000,2023/02/25 16:02:13,1.9,0.4,0.103,0.0,10272.926,Mixed
2023-02-25 16:02:14,47.400251,8.543908,0.002784,291.272282,3.0,3.679465,473.000000,21.360300,30.223839,3.692500,10.371667,16.660000,2023/02/25 16:02:14,2.8,0.4,0.111,0.0,11060.392,Mixed
2023-02-25 16:02:15,47.400255,8.543898,0.003711,286.985333,4.0,3.792503,473.000000,21.329733,30.245687,3.736000,10.535333,16.938000,2023/02/25 16:02:15,3.0,0.4,0.086,0.0,8634.488,Mixed
2023-02-25 16:02:16,47.400260,8.543888,0.004639,282.698385,5.0,3.905541,473.000000,21.299167,30.267534,3.779500,10.699000,17.216000,2023/02/25 16:02:16,3.0,0.0,0.111,0.0,11060.392,Mixed
2023-02-25 16:02:17,47.400265,8.543878,0.005567,278.411436,6.0,4.018579,473.000000,21.268600,30.289382,3.823000,10.862667,17.494000,2023/02/25 16:02:17,3.0,0.0,0.136,0.0,13557.101,Mixed
2023-02-25 16:02:18,47.400269,8.543868,0.006495,274.124487,7.0,4.131618,473.000000,21.238034,30.311229,3.866500,11.026333,17.772000,2023/02/25 16:02:18,3.0,0.0,0.105,0.0,10529.374,Mixed
2023-02-25 16:02:19,47.400274,8.543857,0.007423,269.837538,8.0,4.244656,473.000000,21.207468,30.333076,3.910000,11.190000,18.050000,2023/02/25 16:02:19,2.5,0.4,0.119,0.0,11904.371,Mixed
2023-02-25 16:02:20,47.400267,8.543841,0.008885,265.951461,9.0,4.378094,473.044444,21.177297,30.340216,3.900667,11.132000,17.945167,2023/02/25 16:02:20,2.4,0.8,0.097,0.0,9697.090,Mixed


In [98]:
df_merge.to_csv("20230225_1600_ZH_test_merged.csv", sep=",")